In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Customers who reported to be Defrauded or Hacked - get this list from the Ops team

In [5]:
dfhacked = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\SupportingFile\Database of internal & external fraud incidents v2.xlsx", sheet_name = 'Main')

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
# Replace spaces with underscores in column names
dfhacked.columns = dfhacked.columns.str.replace(' ', '_')
dfhacked.columns = dfhacked.columns.str.replace('/', '_')
dfhacked.columns = dfhacked.columns.str.replace('?', '')

In [9]:
dfhacked.dtypes

Source                           object
Ticket_Number                    object
Ticket_Status                    object
Report_Date              datetime64[ns]
Fraud_Type                       object
Sub_MO                           object
Disputed_Amount                 float64
OFI                              object
RFI                              object
Complainant_Name                 object
Contact_Information              object
Source_Account_Number            object
With_Police_Report               object
TSA_Account_Number              float64
PID                             float64
Account_Name                     object
Segment_KYC                      object
Address1                         object
Address2                         object
Address3                         object
Mobile_Number                   float64
Account_Status                   object
Date_Blocked             datetime64[ns]
TDB_Account_Balance              object
With_TD                          object


In [10]:
dfhacked1 = dfhacked[['Source', 'Ticket_Number', 'Ticket_Status', 'Report_Date', 'Fraud_Type',
       'Sub_MO', 'Disputed_Amount', 'OFI', 'RFI', 'Complainant_Name',
       'Contact_Information', 'With_Police_Report',
       'TSA_Account_Number', 'PID', 'Account_Name', 'Segment_KYC', 'Address1',
       'Address2', 'Address3', 'Mobile_Number', 'Account_Status',
       'Date_Blocked', 'With_TD', 'With_Active_Loans',
       'Callout_Result', 'Disposition', 'Actions']].copy()

In [11]:
from google.cloud import bigquery

# Define the dataset and table references
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'Defraudedandhacked_mu'

# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Source", "STRING"),
    bigquery.SchemaField("Ticket_Number", "STRING"),
    bigquery.SchemaField("Ticket_Status", "STRING"),
    bigquery.SchemaField("Report_Date", "TIMESTAMP"),
    bigquery.SchemaField("Fraud_Type", "STRING"),
    bigquery.SchemaField("Sub_MO", "STRING"),
    bigquery.SchemaField("Disputed_Amount", "FLOAT"),
    bigquery.SchemaField("OFI", "STRING"),
    bigquery.SchemaField("RFI", "STRING"),
    bigquery.SchemaField("Complainant_Name", "STRING"),
    bigquery.SchemaField("Contact_Information", "STRING"),
    bigquery.SchemaField("With_Police_Report", "STRING"),
    bigquery.SchemaField("TSA_Account_Number", "FLOAT"),
    bigquery.SchemaField("PID", "FLOAT"),
    bigquery.SchemaField("Account_Name", "STRING"),
    bigquery.SchemaField("Segment_KYC", "STRING"),
    bigquery.SchemaField("Address1", "STRING"),
    bigquery.SchemaField("Address2", "STRING"),
    bigquery.SchemaField("Address3", "STRING"),
    bigquery.SchemaField("Mobile_Number", "FLOAT"),
    bigquery.SchemaField("Account_Status", "STRING"),
    bigquery.SchemaField("Date_Blocked", "TIMESTAMP"),
    bigquery.SchemaField("With_TD", "STRING"),
    bigquery.SchemaField("With_Active_Loans", "STRING"),
    bigquery.SchemaField("Callout_Result", "STRING"),
    bigquery.SchemaField("Disposition", "STRING"),
    bigquery.SchemaField("Actions", "STRING"),
]

# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.autodetect = False
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Convert datetime columns to timestamp
for col in dfhacked.select_dtypes(include=['datetime64']):
    dfhacked[col] = dfhacked[col].astype('int64') // 10**9

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(dfhacked1, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")

Table Defraudedandhacked_mu created in dataset dap_ds_poweruser_playground.


In [41]:
sq = """              
--drop table if exists `dap_ds_poweruser_playground.demographicscoredata`;
--Create table `dap_ds_poweruser_playground.demographicscoredata` as 
with 
delqtab as 
(select loanAccountNumber,  
case when obs_min_inst_def30 >=2 then 1 else 0 end obsFSPD30,
case when min_inst_def30 in (1,2) then 1 else 0 end fspd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
Excl_evertransacted as 
(select distinct cast(customer_id as numeric) customerId from prj-prod-dataplatform.risk_mart.customer_transactions where status like 'Success'),
Excl_qualified_FSPD30 as
(select loanAccountNumber, obs_min_inst_def30 from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data where obs_min_inst_def30 >= 2),
Excl_blacklisted_clean as 
(select customerid, matching_attribute, id, refresh_date from  prj-prod-dataplatform.CRM_RISK.fraud_blacklist_clean),
educate as 
    (select distinct edu.digitalLoanAccountId, edu.education_id, edu1.description
    from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_purpose` edu
    inner join (select id, description from dl_loans_db_raw.tdbk_loan_lov_mtb where module = 'Education') edu1 on edu.education_id = edu1.id
    ),
    educate2 as 
    (select *, row_number() over(partition by digitalLoanAccountId order by education_id desc) rnk from educate),
    educate3 as 
    (select * from educate2 where rnk = 1),
    education as 
    (
    select 
    lmt.customerId,
    educate3.education_id,
    educate3.description Education_Type, 
    educate3.rnk educationrnk
    from educate3 
    inner join prj-prod-dataplatform.risk_credit_mis.loan_master_table lmt on lmt.digitalLoanAccountId = educate3.digitalLoanAccountId
    ),
    digitalloanapplication as (select 
    digitalLoanAccountId, 
    monthlyInterestRate , 
    interest_rate_offer ,
    monthlyIncomeCheck, 
    from prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application ), 
    loantypeselection as 
    (SELECT 
    digitalLoanAccountId 
    FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where date(disbursementDateTime) >= '2023-07-01'
    and flagDisbursement = 1
    and kycStatus = 'SKYC'
    and customerId in (select customerId from Excl_evertransacted) ---Excluded customers who never transacted
    and loanAccountNumber in (select loanAccountNumber from Excl_qualified_FSPD30) --- Exclude customer who has not yet matured FSPD30
    and customerId not in (select distinct customerid from Excl_blacklisted_clean) --- Exclude customer who are blacklisted
    and customerId not in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) --- Exclude written off
    and customerId not in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null)
    ),
    loan
    as
    (select 
    lmt.digitalLoanAccountId ,
    lmt.customerId ,
    lmt.loanAccountNumber ,
    lmt.startApplyDateTime,
    lmt.decision_date,
    lmt.disbursementDateTime, 
    lmt.new_loan_type,
    lmt.gender,
    lmt.age,
    education.Education_Type,
    education.education_id,
    lmt.custLoanJourneyCity,
    lmt.custLoanJourneyProvince,
    lmt.custLoanJourneyZipCode,
    lmt.province,
    lmt.city,
    lmt.postalCode,
    lmt.incomeperSalaryScale,
    lmt.monthlyIncome,
    lmt.maritalStatus,
    lmt.purposeDescription,
    lmt.isfatca,
    lmt.docType,
    lmt.loanRequestAmount,
    lmt.loanRequestTenure,
    dlp.monthlyInterestRate,
    dlp.interest_rate_offer,
    dlp.monthlyIncomeCheck,
    lmt.kycStatus,
	case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then 'ios'
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then 'android'
		 else 'other' end Device_os_type,
    delqtab.obsFSPD30,
    delqtab.FSPD30
	-- case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 4, 2)  as numeric),0)
	--      when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0)
	-- 	 else 'NA' end Device_OSVersion
    from `risk_credit_mis.loan_master_table` lmt
    inner join loantypeselection lts on lts.digitalLoanAccountId = lmt.digitalLoanAccountId 
    inner join delqtab on delqtab.loanAccountNumber = lmt.loanAccountNumber
    Left join education on education.customerId = lmt.customerId
    left join digitalloanapplication dlp on dlp.digitalLoanAccountId = lmt.digitalLoanAccountId
    )
    select * from loan;
      
"""

In [43]:
%%time

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 11a0669a-f88c-4539-8578-bf8b2f6056d0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
CPU times: total: 7.58 s
Wall time: 49.7 s


In [44]:
df.head()

,digitalLoanAccountId,customerId,loanAccountNumber,startApplyDateTime,decision_date,disbursementDateTime,new_loan_type,gender,age,Education_Type,...,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,obsFSPD30,FSPD30
0,5e3a01d8-ee89-4589-a873-6b909030949f,2407782,60824077820017,2024-02-22 11:12:59,2024-02-22 11:15:44,2024-02-22 11:36:08,SIL-Instore,F,33,College Graduate,...,Philippines - Id Card (2020),20000.0,12,4.50,None,true,SKYC,android,1,0
1,f80a0c8c-534b-4e0e-bd52-ff3b6f221131,2600438,60826004380018,2024-06-22 15:07:52,2024-06-22 15:13:24,2024-06-22 15:36:18,SIL-Instore,F,42,High School Graduate,...,Philippines - Id Card (2020),14959.0,12,5.08,None,None,SKYC,android,1,0
2,35c087d7-f2fd-47c2-8628-96644bf2d6b0,2518334,60825183340012,2024-05-16 18:00:17,2024-05-16 18:03:27,2024-05-16 18:19:24,SIL-Instore,F,23,High School Graduate,...,Philippines - Id Card (2020),6989.0,8,6.62,None,false,SKYC,android,1,0
3,ca27adf9-ef13-455b-8d94-f79dc5810f12,2431151,60824311510013,2024-03-16 14:10:00,2024-03-16 14:14:18,2024-03-16 14:56:23,SIL-Instore,F,50,College Graduate,...,Philippines - UMID Card (2016),14109.0,12,4.50,None,true,SKYC,android,1,0
4,45a52c4e-3cd7-4ba0-8a03-4b8864ac66c8,2278610,60822786100015,2023-10-20 13:04:39,2023-10-20 13:08:38,2023-10-20 13:13:24,SIL-Instore,F,23,High School Graduate,...,Philippines - Driving License,6598.0,9,4.50,None,true,SKYC,android,1,0


In [45]:
df.dtypes

digitalLoanAccountId               object
customerId                          Int64
loanAccountNumber                  object
startApplyDateTime         datetime64[us]
decision_date              datetime64[us]
disbursementDateTime       datetime64[us]
new_loan_type                      object
gender                             object
age                                 Int64
Education_Type                     object
education_id                       object
custLoanJourneyCity                object
custLoanJourneyProvince            object
custLoanJourneyZipCode             object
province                           object
city                               object
postalCode                         object
incomeperSalaryScale               object
monthlyIncome                      object
maritalStatus                      object
purposeDescription                 object
isfatca                            object
docType                            object
loanRequestAmount                 

In [46]:
print(f"The shape of the dataframe is:\t {df.shape}")

The shape of the dataframe is:	 (101230, 32)


In [47]:
df.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\Demo_Telco_Data_20241008_allsil.csv", index = False)

In [48]:
df['new_loan_type'].value_counts()

new_loan_type
SIL-Instore    74512
Quick          16929
Flex-up         8586
Flex            1191
ACL TSA           11
Big Loan           1
Name: count, dtype: int64

# Bala's Query

In [20]:
sq = """select  distinct customerId,user_id,tsaOnboardingDateTime,credolab_ReferenceNumber,deviceType,
digitalLoanAccountId,case when loan_type in ('Flex-up','Reloan') then 'Repeat Applicant' 
when First_application=1 and customer_id_first_applicant is not null then 'New Applicant'
when accept_cust_id is null then  'Prev_Rejected_Reapply'
when rn=1 then 'New Applicant' 
--when Reject_application_count=1 then 'New Applicant' 
when accept_application_count>1 and accept_cust_id is not null then  'Repeat Applicant'

else 'Repeat Applicant' 
end as applicant_type,loan_type,loan_sub_type,sil_product,application_Submission_DateTime,
fraudScore,credolabScore,decision,applicationStatus,disbursementFlag,flg_mature_fpd10,flg_mature_fpd30,
flg_mature_fspd_30,flg_mature_fstpd_30,deffpd10,deffpd30,deffspd30,deffstpd30

from (with deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

accept_application_cnt as (select customerId,count(*) as accept_application_count
from risk_credit_mis.loan_master_table loan where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null and applicationStatus in ('ACTIVATED','COMPLETED','APPROVED',
'INPROGRESS','SIGNEDDOCCOMPLETED','ACCEPT') group by 1 order by customerId),


applciant_type_first as (select customerId customer_id_first_applicant,count(*) First_application from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
where new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or new_loan_type in ('Flex-up')) and coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))  is not null group by 1 having count(*)=1),

previous_loan_product as (select loan.customerId,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
     WHEN loan.new_loan_type ='Flex' then 'Flex'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as prev_new_loan_type,loan.loanType prev_loan_type,prev_loan.previous_loan_acct,prev_loan.loanAccountNumber,loan.startApplyDateTime from `risk_credit_mis.loan_master_table` loan
join
(select * from (select customerId, digitalLoanAccountId,loanAccountNumber,new_loan_type,startApplyDateTime, lead(loanAccountNumber)
 OVER(PARTITION BY customerId order by startApplyDateTime desc) as previous_loan_acct from `risk_credit_mis.loan_master_table`
 where applicationStatus in ('COMPLETED','ACTIVATED')  order by customerId) where --previous_loan_acct is not null and 
 new_loan_type='Flex-up')prev_loan on loan.loanAccountNumber=prev_loan.previous_loan_acct)
  
  select
  loan.customerId,
  cust.user_id,
  tsa_onboarding_time as tsaOnboardingDateTime,
  credolabRefNumber as credolab_ReferenceNumber,
  deviceType,
  loan.digitalLoanAccountId,
  --case when applciant_type_first.disbursed_loan >1 then 'Repeat Applicant' else 'New Applicant' end as applicant_type,

    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as loan_type,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then coalesce('Reloan_'||prev_new_loan_type)
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then coalesce('Flex-up_'||prev_new_loan_type)
    else loan.new_loan_type end loan_sub_type,
    
    --new_loan_type as sub_loan_type,reloan_flag,
    --prev_new_loan_type, 
    case when loan.loantype='BNPL' and store_type =1 then 'Appliance'
    when loan.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loan.loantype='BNPL' and store_type =3 then 'Mall' 
    when loan.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as sil_product,
    coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime)) application_Submission_DateTime,
    fraudScore,credolabScore,
      case
      when flagApproval = 1 then 'approved'
      when flagRejection = 1 then 'rejected'
      else 'auto-approved without model'
    end as decision,
    applicationStatus,
    flagDisbursement as disbursementFlag,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 40, 1 ,0) as flg_mature_fpd10,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 60, 1 ,0) as flg_mature_fpd30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 90, 1 ,0) as flg_mature_fspd_30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 120, 1 ,0) as flg_mature_fstpd_30,
    deliquency.deffpd10,deliquency.deffpd30,deliquency.deffspd30,deliquency.deffstpd30,row_number() over (partition by loan.customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',loan.startApplyDateTime,termsAndConditionsSubmitDateTime))) rn,accept_application_count,accept_application_cnt.customerId accept_cust_id,First_application,customer_id_first_applicant

   
  from prj-prod-dataplatform.risk_credit_mis.loan_master_table loan
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name) sil_category on purpleKey=sil_category.mer_refferal_code
  left join deliquency on deliquency.loanAccountNumber=loan.loanAccountNumber
  left join previous_loan_product on previous_loan_product.loanAccountNumber=loan.loanAccountNumber
  left join applciant_type_first on applciant_type_first.customer_id_first_applicant=loan.customerId
  left join dl_customers_db_raw.tdbk_customer_mtb cust on cust.cust_id=cast(loan.customerId as string)
  left join accept_application_cnt on accept_application_cnt.customerId=loan.customerId
  where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null) final where loan_sub_type is not null;"""

In [21]:
dfbala = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1970e491-e9c7-482b-8a69-f39e11bef974 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [22]:
dfbala.shape

(1033025, 24)

In [23]:
dfbala.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\bala.csv", index = False)

In [33]:
dfbala[(dfbala['flg_mature_fspd_30']==1)&(dfbala['loan_type'] == 'SIL')].groupby(['deviceType', 'deffspd30'])['digitalLoanAccountId'].count()

deviceType  deffspd30
Android     0            62989
            1             8173
iOS         0             8233
            1              923
iPadOS      0               23
            1                7
Name: digitalLoanAccountId, dtype: int64

In [59]:
sq = """
with base as 
(select * from
(select  distinct customerId,user_id,tsaOnboardingDateTime,credolab_ReferenceNumber,deviceType,
digitalLoanAccountId,case when loan_type in ('Flex-up','Reloan') then 'Repeat Applicant' 
when First_application=1 and customer_id_first_applicant is not null then 'New Applicant'
when accept_cust_id is null then  'Prev_Rejected_Reapply'
when rn=1 then 'New Applicant' 
--when Reject_application_count=1 then 'New Applicant' 
when accept_application_count>1 and accept_cust_id is not null then  'Repeat Applicant'

else 'Repeat Applicant' 
end as applicant_type,loan_type,loan_sub_type,sil_product,application_Submission_DateTime,
fraudScore,credolabScore,decision,applicationStatus,disbursementFlag,flg_mature_fpd10,flg_mature_fpd30,
flg_mature_fspd_30,flg_mature_fstpd_30,deffpd10,deffpd30,deffspd30,deffstpd30

from (with deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

accept_application_cnt as (select customerId,count(*) as accept_application_count
from risk_credit_mis.loan_master_table loan where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null and applicationStatus in ('ACTIVATED','COMPLETED','APPROVED',
'INPROGRESS','SIGNEDDOCCOMPLETED','ACCEPT') group by 1 order by customerId),


applciant_type_first as (select customerId customer_id_first_applicant,count(*) First_application from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
where new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or new_loan_type in ('Flex-up')) and coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))  is not null group by 1 having count(*)=1),

previous_loan_product as (select loan.customerId,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
     WHEN loan.new_loan_type ='Flex' then 'Flex'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as prev_new_loan_type,loan.loanType prev_loan_type,prev_loan.previous_loan_acct,prev_loan.loanAccountNumber,loan.startApplyDateTime from `risk_credit_mis.loan_master_table` loan
join
(select * from (select customerId, digitalLoanAccountId,loanAccountNumber,new_loan_type,startApplyDateTime, lead(loanAccountNumber)
 OVER(PARTITION BY customerId order by startApplyDateTime desc) as previous_loan_acct from `risk_credit_mis.loan_master_table`
 where applicationStatus in ('COMPLETED','ACTIVATED')  order by customerId) where --previous_loan_acct is not null and 
 new_loan_type='Flex-up')prev_loan on loan.loanAccountNumber=prev_loan.previous_loan_acct)
  
  select
  loan.customerId,
  cust.user_id,
  tsa_onboarding_time as tsaOnboardingDateTime,
  credolabRefNumber as credolab_ReferenceNumber,
  deviceType,
  loan.digitalLoanAccountId,
  --case when applciant_type_first.disbursed_loan >1 then 'Repeat Applicant' else 'New Applicant' end as applicant_type,

    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as loan_type,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then coalesce('Reloan_'||prev_new_loan_type)
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then coalesce('Flex-up_'||prev_new_loan_type)
    else loan.new_loan_type end loan_sub_type,
    
    --new_loan_type as sub_loan_type,reloan_flag,
    --prev_new_loan_type, 
    case when loan.loantype='BNPL' and store_type =1 then 'Appliance'
    when loan.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loan.loantype='BNPL' and store_type =3 then 'Mall' 
    when loan.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as sil_product,
    coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime)) application_Submission_DateTime,
    fraudScore,credolabScore,
      case
      when flagApproval = 1 then 'approved'
      when flagRejection = 1 then 'rejected'
      else 'auto-approved without model'
    end as decision,
    applicationStatus,
    flagDisbursement as disbursementFlag,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 40, 1 ,0) as flg_mature_fpd10,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 60, 1 ,0) as flg_mature_fpd30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 90, 1 ,0) as flg_mature_fspd_30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 120, 1 ,0) as flg_mature_fstpd_30,
    deliquency.deffpd10,deliquency.deffpd30,deliquency.deffspd30,deliquency.deffstpd30,row_number() over (partition by loan.customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',loan.startApplyDateTime,termsAndConditionsSubmitDateTime))) rn,accept_application_count,accept_application_cnt.customerId accept_cust_id,First_application,customer_id_first_applicant

   
  from prj-prod-dataplatform.risk_credit_mis.loan_master_table loan
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name) sil_category on purpleKey=sil_category.mer_refferal_code
  left join deliquency on deliquency.loanAccountNumber=loan.loanAccountNumber
  left join previous_loan_product on previous_loan_product.loanAccountNumber=loan.loanAccountNumber
  left join applciant_type_first on applciant_type_first.customer_id_first_applicant=loan.customerId
  left join dl_customers_db_raw.tdbk_customer_mtb cust on cust.cust_id=cast(loan.customerId as string)
  left join accept_application_cnt on accept_application_cnt.customerId=loan.customerId
  where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null) final where loan_sub_type is not null)
), 
delqtab as 
(select loanAccountNumber,  
case when obs_min_inst_def30 >=2 then 1 else 0 end obsFSPD30,
case when min_inst_def30 in (1,2) then 1 else 0 end fspd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
Excl_evertransacted as 
(select distinct cast(customer_id as numeric) customerId from prj-prod-dataplatform.risk_mart.customer_transactions where status like 'Success'),
Excl_qualified_FSPD30 as
(select loanAccountNumber, obs_min_inst_def30 from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data where obs_min_inst_def30 >= 2),
Excl_blacklisted_clean as 
(select customerid, matching_attribute, id, refresh_date from  prj-prod-dataplatform.CRM_RISK.fraud_blacklist_clean),
educate as 
    (select distinct edu.digitalLoanAccountId, edu.education_id, edu1.description
    from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_purpose` edu
    inner join (select id, description from dl_loans_db_raw.tdbk_loan_lov_mtb where module = 'Education') edu1 on edu.education_id = edu1.id
    ),
    educate2 as 
    (select *, row_number() over(partition by digitalLoanAccountId order by education_id desc) rnk from educate),
    educate3 as 
    (select * from educate2 where rnk = 1),
    education as 
    (
    select 
    lmt.customerId,
    educate3.digitalLoanAccountId,
    educate3.education_id,
    educate3.description Education_Type, 
    educate3.rnk educationrnk
    from educate3 
    inner join prj-prod-dataplatform.risk_credit_mis.loan_master_table lmt on lmt.digitalLoanAccountId = educate3.digitalLoanAccountId
    )
    ,
    digitalloanapplication as (select 
    digitalLoanAccountId, 
    monthlyInterestRate , 
    interest_rate_offer ,
    monthlyIncomeCheck, 
    from prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application ), 
    loantypeselection as 
    (SELECT 
    digitalLoanAccountId 
    FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where date(disbursementDateTime) >= '2023-07-01'
    and flagDisbursement = 1
    and kycStatus = 'SKYC'
    -- and customerId in (select customerId from Excl_evertransacted) ---Excluded customers who never transacted
    -- and loanAccountNumber in (select loanAccountNumber from Excl_qualified_FSPD30) --- Exclude customer who has not yet matured FSPD30
    -- and customerId not in (select distinct customerid from Excl_blacklisted_clean) --- Exclude customer who are blacklisted
    -- and customerId not in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) --- Exclude written off
    -- and customerId not in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null)
    ),
    Appreinstall as 
    (select cast(customer_id as numeric) customerId, count(customer_id)  from moengage_raw.events_hourly where event_code = 'REINSTALL'
        group by 1
        having count(customer_id) > 1)
    ,
    loan
    as
    (select 
    distinct
    lmt.digitalLoanAccountId ,
    lmt.customerId ,
    lmt.loanAccountNumber ,
    lmt.tsa_onboarding_time,
    lmt.startApplyDateTime,
    lmt.decision_date,
    lmt.termsAndConditionsSubmitDateTime,
    lmt.disbursementDateTime, 
    lmt.new_loan_type,
    lmt.gender,
    lmt.age,
    education.Education_Type,
    education.education_id,
    lmt.custLoanJourneyCity,
    lmt.custLoanJourneyProvince,
    lmt.custLoanJourneyZipCode,
    lmt.province,
    lmt.city,
    lmt.postalCode,
    lmt.incomeperSalaryScale,
    lmt.monthlyIncome,
    lmt.maritalStatus,
    lmt.purposeDescription,
    lmt.isfatca,
    lmt.docType,
    lmt.loanRequestAmount,
    lmt.loanRequestTenure,
    dlp.monthlyInterestRate,
    dlp.interest_rate_offer,
    dlp.monthlyIncomeCheck,
    lmt.kycStatus,
	case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then 'ios'
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then 'android'
		 else 'other' end Device_os_type,
    delqtab.obsFSPD30,
    delqtab.FSPD30,
	  case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 4, 2)  as numeric),0)
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0)
		 end Device_OSVersion,
    b.Network Network_provider,
	case when eet.customerId is null then 1 else 0 end customer_never_transacted_flag,
	case when eqf.loanAccountNumber is not null then 1 else 0 end fspd30_mature_flag,
	case when ebc.customerId is not null then 1 else 0 end blacklisted_customer_flag,
	case when lmt.customerId in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) then 1 else 0 end writtenoff_flag,
	case when lmt.customerId in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null) then 1 else 0 end hackedordefrauded_flag,
  case when e.vas_flag = 'true' then 1 else 0 end as VasFlag,
  case when Appreinstall.customerId is not null then 1 else 0 end EverReinstallFlag
    from base 
    left join   `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = base.digitalLoanAccountId
    inner join loantypeselection lts on lts.digitalLoanAccountId = lmt.digitalLoanAccountId 
    inner join delqtab on delqtab.loanAccountNumber = lmt.loanAccountNumber
    Left join education on education.digitalLoanAccountId = lmt.digitalLoanAccountId
    left join digitalloanapplication dlp on dlp.digitalLoanAccountId = lmt.digitalLoanAccountId
    LEFT JOIN (select *, row_number() over(partition by Number_Prefix order by Network) rnk 
  from `prj-prod-dataplatform.manual_source_extracts.mobile_carrier_mapping`) b ON LEFT(RIGHT(lmt.mobileNo,LENGTH(lmt.mobileNo)-2),3) = CAST(b.number_prefix AS STRING)
	 left join Excl_evertransacted eet on eet.customerId = lmt.customerId
	 left join Excl_qualified_FSPD30 eqf on eqf.loanAccountNumber = lmt.loanAccountNumber
	 left join Excl_blacklisted_clean ebc on ebc.customerId = lmt.customerId
   left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e
          on e.loanAccountNumber = lmt.loanAccountNumber
   left join Appreinstall on Appreinstall.customerId = lmt.customerId
    where b.rnk = 1
    )
    select * from loan ;


    """

In [60]:
dfbalame = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {dfbalame.shape}")

Job ID 74c0adb5-5481-4f99-85b5-df64855a1c18 successfully executed: |          | 
Downloading: 100%|██████████|
The shape of the dataframe is:	 (128889, 43)


In [61]:
dfbalame.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\demographicdatabalamecombined20241009_all.csv", index = False)